Análise Comparativa de Dados

In [14]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from sklearn.linear_model import LinearRegression

1. Obtenção de dados

In [15]:
df = pd.read_csv('../data/raw/Churn_Modelling.csv')
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,RowNumber,O número sequencial atribuído a cada linha,Inutil,NaN
1,CustomerId,Identificador único do cliente,Quantitativa,Discreta
2,Surname,Sobrenome do Cliente,Inutil,NaN
3,CreditScore,Pontuação de crédito do cliente,Quantitativa,Discreta
4,Geography,Localização geográfica do cliente,Qualitativa,Nominal
5,Gender,Genero do Cliente,Qualitativa,Nominal
6,Age,Idade do Cliente,Quantitativa,Discreta
7,Tenure,O número de anos que o cliente está no banco,Quantitiva,Discreta
8,Balance,Saldo da conta do cliente,Quantitativa,Continua
9,NumOfProducts,Número de produtos bancários que o cliente possui,Quantitativa,Discreta


2. Tratamento de dados faltantes

In [28]:
print(f'A quantidades de dado null é: {df.isnull().sum().sum()}')
print(f'A quantidades de dado NAN é: {df.isna().sum().sum()}')

A quantidades de dado null é: 0
A quantidades de dado NAN é: 0


3. Normalização de Dados

In [ ]:
target_column = 'Exited'
useless_variables =  (
    df_dict
    .query("tipo == 'Inutil'")
    .variavel
    .to_list()
)
nominal_variables = (
    df_dict
    .query("subtipo == 'Nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)
ordinal_variables = (
    df_dict
    .query("subtipo == 'Ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)
continuous_variables = (
    df_dict
    .query("subtipo == 'Continua' and variavel != @target_variable")
    .variavel
    .to_list()
)
discrete_variables = (
    df_dict
    .query("subtipo == 'Discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)


In [23]:
list(df.columns)

['RowNumber',
 'CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

In [30]:
nominal_preprocessor = Pipeline(steps=[ 
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False)),
])
continuous_preprocessor = Pipeline(steps=[ 
    ("missing", SimpleImputer(strategy='median')),
    ("normalization", OrdinalEncoder()),
])
discrete_preprocessor = Pipeline(steps=[ 
    ("missing", SimpleImputer(strategy='median')),
    ("normalization", OrdinalEncoder()),
])

In [31]:
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),
    ("continuous", continuous_preprocessor, continuous_variables),
    ("discrete", discrete_preprocessor, discrete_variables)  
])